In [1]:
import pandas as pd
import numpy as np
import glob
 
import os
import zipfile

import tensorflow as tf

from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Activation
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import load_model

In [2]:
# 다운로드 받은 이미지 압축 파일 해제
# print(os.listdir("./drive/MyDrive"))
local_zip = './train.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./res/train')
zip_ref.close()

local_zip = './test.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./res/test')
zip_ref.close()

In [3]:
base_dir = "./res/"

train_dir = "train/train/"

train_class = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']


test_dir = "test/test/0"

In [4]:
train_one_hot = []

for path in range(len(train_class)):
    for img in os.listdir(base_dir + train_dir + train_class[path]):
        class2label = [''] + [0] * len(train_class)
        class2label[0] = (train_dir + train_class[path] + "/" + img)[6:]
        class2label[path + 1] = 1
        train_one_hot.append(class2label)

        
train_answer = pd.DataFrame(train_one_hot, columns = ["path", "dogs", 'elephant', 'giraffe', 'guitar', 'horse' , 'house', 'person'])
train_answer.to_csv('./train_answer.csv', index=False)

data = pd.read_csv("./train_answer.csv")
columns = data.columns

datagen=ImageDataGenerator(
      rescale = 1/255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator=datagen.flow_from_dataframe(
                                            dataframe=data,
                                            directory='./res/train/',
                                            x_col="path",
                                            y_col=columns[1:],
                                            batch_size=32,
                                            shuffle=False,
                                            class_mode="raw",
                                            target_size=(299,299))
data

Found 1698 validated image filenames.


,path,dogs,elephant,giraffe,guitar,horse,house,person
0,train/dog/pic_327.jpg,1,0,0,0,0,0,0
1,train/dog/pic_064.jpg,1,0,0,0,0,0,0
2,train/dog/pic_037.jpg,1,0,0,0,0,0,0
3,train/dog/pic_310.jpg,1,0,0,0,0,0,0
4,train/dog/pic_167.jpg,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1693,train/person/pic_337.jpg,0,0,0,0,0,0,1
1694,train/person/pic_160.jpg,0,0,0,0,0,0,1
1695,train/person/pic_429.jpg,0,0,0,0,0,0,1
1696,train/person/pic_059.jpg,0,0,0,0,0,0,1


In [5]:
xception = tf.keras.applications.Xception(
    include_top=True, weights='imagenet',input_shape=(299,299,3), pooling="max")
xception.trainable = True # 가중치 학습 여부 설정 (False : 가중치 학습 X)
xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [6]:
pretrained_data = xception.predict(train_generator, verbose=1)

x_train, x_valid, y_train, y_valid = train_test_split(pretrained_data, data.iloc[:,1:], test_size=0.2, random_state=42)

54/54 [==============================] - 55s 769ms/step


In [7]:
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential(
    [Input(1000,),
     
    Dropout(0.2),
    
    Dense(256,  activation='relu'),
    
    Dense(128,  activation='relu'),
    
    Dense(7,  activation='softmax')
    ])
model.summary()
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

mc = tf.keras.callbacks.ModelCheckpoint('xception_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               256256    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 903       
Total params: 290,055
Trainable params: 290,055
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy']) # 최적화 함수 학습률 1e-4에서 0.001로 변경
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_valid, y_valid),callbacks=[es,mc])

Epoch 1/100
43/43 [==============================] - 1s 7ms/step - loss: 1.9405 - accuracy: 0.2518 - val_loss: 1.9361 - val_accuracy: 0.2853

Epoch 00001: val_loss improved from inf to 1.93608, saving model to xception_model.h5
Epoch 2/100
43/43 [==============================] - 0s 4ms/step - loss: 1.9271 - accuracy: 0.3078 - val_loss: 1.9243 - val_accuracy: 0.2824

Epoch 00002: val_loss improved from 1.93608 to 1.92430, saving model to xception_model.h5
Epoch 3/100
43/43 [==============================] - 0s 4ms/step - loss: 1.9097 - accuracy: 0.3255 - val_loss: 1.9084 - val_accuracy: 0.2882

Epoch 00003: val_loss improved from 1.92430 to 1.90836, saving model to xception_model.h5
Epoch 4/100
43/43 [==============================] - 0s 4ms/step - loss: 1.8867 - accuracy: 0.3233 - val_loss: 1.8885 - val_accuracy: 0.2853

Epoch 00004: val_loss improved from 1.90836 to 1.88847, saving model to xception_model.h5
Epoch 5/100
43/43 [==============================] - 0s 4ms/step - loss: 1.8

In [9]:
loaded_model = load_model('xception_model.h5')
 
final_model = Sequential([xception, model])
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 1000)              22910480  
_________________________________________________________________
sequential (Sequential)      (None, 7)                 290055    
Total params: 23,200,535
Trainable params: 23,146,007
Non-trainable params: 54,528
_________________________________________________________________


In [10]:
test_df = pd.DataFrame()
test = glob.glob("./res/test/test/0/*.jpg")
test.sort()
test_dir = []
for img in test:
    test_dir.append(img[11:])
test_df["path"] = test_dir
 
test_datagen=ImageDataGenerator(rescale = 1/255)
test_generator = test_datagen.flow_from_dataframe(  dataframe=test_df[:],
                                                    directory='./res/test/',
                                                    x_col="path",
                                                    y_col=columns[0],
                                                    batch_size=1,
                                                    shuffle=False,
                                                    class_mode="raw",
                                                    target_size=(299,299))

Found 350 validated image filenames.


In [11]:
pred = final_model.predict(test_generator, verbose=1)

350/350 [==============================] - 18s 14ms/step


In [12]:
answer = np.array([y.argmax() for y in pred])
test_df = pd.read_csv("./test_answer_sample_.csv")
test_df.iloc[:,1] = answer
test_df

,Unnamed: 0,answer value
0,0,2
1,1,3
2,2,3
3,3,3
4,4,6
...,...,...
345,345,1
346,346,6
347,347,3
348,348,6


In [13]:
test_df.to_csv('xception_50.csv', index=False)